In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings("ignore")

# 设置随机种子以保证结果可复现
torch.manual_seed(42)
np.random.seed(42)

# 定义神经网络模型
class MultiClassifier(nn.Module):
    def __init__(self, input_size):
        super(MultiClassifier, self).__init__()
        # 第一层使用较大的隐藏层，模拟树模型的特征组合能力
        self.layer1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.3)
        
        # 第二层
        self.layer2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.2)
        
        # 输出层
        self.layer3 = nn.Linear(64, 2)  # 二分类问题，输出为2
    
    def forward(self, x):
        # 使用ELU激活函数，与常见的ReLU相比提供更平滑的激活
        x = torch.elu(self.layer1(x))
        x = self.bn1(x)
        x = self.dropout1(x)
        
        x = torch.elu(self.layer2(x))
        x = self.bn2(x)
        x = self.dropout2(x)
        
        x = self.layer3(x)
        return x

# 读取数据
dataTrain = pd.read_csv("allAtt_onehot_large_train_new8.csv")
dataTest = pd.read_csv("allAtt_onehot_large_test_new8.csv")

# 准备数据
x_train, y_train = dataTrain.iloc[:,4:38].values, dataTrain.iloc[:,38:].values
x_test, y_test = dataTest.iloc[:,4:38].values, dataTest.iloc[:,38:].values

# 如果 y 是 one-hot 编码，则转为整数标签
y_train_int = np.argmax(y_train, axis=1)
y_test_int = np.argmax(y_test, axis=1)

# 转换为PyTorch张量
X_train_tensor = torch.FloatTensor(x_train)
y_train_tensor = torch.LongTensor(y_train_int)
X_test_tensor = torch.FloatTensor(x_test)
y_test_tensor = torch.LongTensor(y_test_int)

# 创建数据加载器
batch_size = 64  # 增加批量大小以加速训练
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# 初始化模型
input_size = x_train.shape[1]  # 特征数量
model = MultiClassifier(input_size)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)  # 使用AdamW优化器，添加正则化
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

# 训练模型
num_epochs = 100
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# 创建验证集进行早停
val_size = int(len(X_train_tensor) * 0.1)
train_indices = list(range(len(X_train_tensor) - val_size))
val_indices = list(range(len(X_train_tensor) - val_size, len(X_train_tensor)))

train_subsampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_subsampler = torch.utils.data.SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=train_subsampler)
val_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, sampler=val_subsampler)

# 早停机制
best_val_loss = float('inf')
patience = 10
patience_counter = 0

for epoch in range(num_epochs):
    # 训练阶段
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # 清零梯度
        optimizer.zero_grad()
        
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss = train_loss / len(train_loader)
    
    # 验证阶段
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    val_loss = val_loss / len(val_loader)
    
    # 学习率调整
    scheduler.step(val_loss)
    
    # 早停检查
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # 保存最佳模型
        torch.save(model.state_dict(), 'models/pytorch_lightgbm_model.pth')
    else:
        patience_counter += 1
    
    # 打印训练进度
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # 如果连续patience次没有改善，则停止训练
    if patience_counter >= patience:
        print(f'Early stopping at epoch {epoch+1}')
        break

# 加载最佳模型
model.load_state_dict(torch.load('models/pytorch_lightgbm_model.pth'))

# 评估模型
model.eval()
with torch.no_grad():
    X_test_tensor = X_test_tensor.to(device)
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs.data, 1)
    predicted = predicted.cpu().numpy()

# 计算准确率
acc = accuracy_score(y_test_int, predicted)
print(f"✅ Accuracy: {acc:.4f}")
print("📊 Classification Report:")
print(classification_report(y_test_int, predicted))

# 模型保存/加载函数
def save_model(model, path):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

def load_model(model_path, input_size):
    model = MultiClassifier(input_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# 预测函数
def predict(model, X):
    model.eval()
    with torch.no_grad():
        X_tensor = torch.FloatTensor(X).to(next(model.parameters()).device)
        outputs = model(X_tensor)
        _, predicted = torch.max(outputs.data, 1)
        return predicted.cpu().numpy()

# 使用示例：
# 1. 保存模型
# save_model(model, 'models/pytorch_lightgbm_model.pth')

# 2. 加载模型并预测
# loaded_model = load_model('models/pytorch_lightgbm_model.pth', input_size)
# predictions = predict(loaded_model, x_test)